## Training

In [8]:
import torchvision
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(5531)

# Prepare trainig data
train_data = torchvision.datasets.SVHN('./files/', 
                                  split='test', #Tak.
                                  download=True)

train_X = (torch.tensor(train_data.data)/255).to(torch.float)
train_y = train_data.labels

dataset = TensorDataset(torch.Tensor(train_X),torch.Tensor(train_y))
dataloader = DataLoader(dataset, batch_size=64)

print(train_X.shape)
print(train_y.shape)

Using downloaded and verified file: ./files/test_32x32.mat
torch.Size([26032, 3, 32, 32])
(26032,)


In [2]:
from methods.architectures import *

# Initialize and train classifiers
max_training_epochs = 750
training_support_level = 0.9

clfs = [
    CNN(architecure=CNN1_5_Network(img_depth=3, x_input_size=32)),
    CNN(architecure=CNN2_5_10_Network(img_depth=3, x_input_size=32)),
    CNN(architecure=CNN2_10_15_Network(img_depth=3, x_input_size=32)),
    CNN(architecure=CNN2_20_30_Network(img_depth=3, x_input_size=32)),
    CNN(architecure=CNN2_25_40_Network(img_depth=3, x_input_size=32)),
]

loss_fn = torch.nn.CrossEntropyLoss()

for clf_id, clf in enumerate(clfs):
    optimizer = torch.optim.SGD(clf.parameters(), lr=1e-2)

    for e in range(max_training_epochs):
        if e==0:
            clf.custom_train(dataloader, loss_fn, optimizer)
        else:

            proba = nn.Softmax(dim=1)(clf(train_X))
            max_proba = torch.max(proba, dim=1)[0] 
            mean_proba = torch.mean(max_proba).detach().numpy() # średnie wsparcie decyzyjne

            if mean_proba>training_support_level:
                print(clf_id, e, mean_proba)
                break
            
            clf.custom_train(dataloader, loss_fn, optimizer)
        
        if e%50==1:
            print(clf_id, e, mean_proba)
            

0 1 0.1914854
0 51 0.7555662
0 101 0.78944904
0 151 0.8039864
0 201 0.81441087
0 251 0.82296103
0 301 0.8299477
0 351 0.8350868
0 401 0.83927864
0 451 0.8424678
0 501 0.8451011
0 551 0.8473635
0 601 0.8492799
0 651 0.85096747
0 701 0.8524579
1 1 0.18801624
1 51 0.8362857
1 101 0.85830474
1 151 0.8693057
1 201 0.8749631
1 251 0.8786195
1 301 0.88175875
1 351 0.8840639
1 401 0.88642263
1 451 0.8885215
1 501 0.89071083
1 551 0.8924491
1 601 0.8932621
1 651 0.8945484
1 701 0.8959729
2 1 0.18699118
2 51 0.85794234
2 101 0.87988263
2 151 0.8895973
2 201 0.89564013
2 249 0.90001243
3 1 0.18811534
3 51 0.87535757
3 95 0.900252
4 1 0.18955737
4 51 0.890464
4 64 0.90029675


In [3]:
from sklearn.metrics import accuracy_score
import time 

# Test on training data
for c_id, c in enumerate(clfs):
    st = time.time()
    proba = nn.Softmax(dim=1)(c(train_X))
    p = torch.argmax(proba, dim=1)    
    el = time.time() - st
    
    print(c_id, accuracy_score(train_y, p), el)
                        

0 0.8822218807621389 0.8646008968353271
1 0.9147587584511371 0.7050721645355225
2 0.9153733866011063 1.3436081409454346
3 0.9169099569760295 3.3284401893615723
4 0.9228257529194838 4.940171957015991
